In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

movies_df = pd.read_csv('movies.csv')
genres_list = movies_df['genres'].str.get_dummies(sep='|')
content_features = genres_list.values

content_cosine_sim = cosine_similarity(content_features, content_features)

ratings_matrix = np.random.randint(1, 6, size=(len(movies_df), len(movies_df)))

mean_ratings = np.mean(ratings_matrix, axis=1, keepdims=True)
centered_ratings_matrix = ratings_matrix - mean_ratings

collab_cosine_sim = cosine_similarity(centered_ratings_matrix, centered_ratings_matrix)

def get_top_similar_movies(similarity_matrix, movie_index, n=5):
    movie_similarities = similarity_matrix[movie_index]
    similar_indices = np.argsort(movie_similarities)[::-1]
    similar_indices = similar_indices[similar_indices != movie_index]
    return similar_indices[:n], movie_similarities[similar_indices[:n]]

movie_index = 0
similar_movies, similarity_scores = get_top_similar_movies(content_cosine_sim, movie_index)
print("Content-based filtering:")
print("Similar movies to '{}':".format(movies_df.iloc[movie_index]['title']))
for movie_idx, similarity_score in zip(similar_movies, similarity_scores):
    print("{}, Similarity Score: {}".format(movies_df.iloc[movie_idx]['title'], similarity_score))

similar_movies, similarity_scores = get_top_similar_movies(collab_cosine_sim, movie_index)
print("\nCollaborative-based filtering:")
print("Similar movies to '{}':".format(movies_df.iloc[movie_index]['title']))
for movie_idx, similarity_score in zip(similar_movies, similarity_scores):
    print("{}, Similarity Score: {}".format(movies_df.iloc[movie_idx]['title'], similarity_score))

Content-based filtering:
Similar movies to 'Toy Story (1995)':
Turbo (2013), Similarity Score: 0.9999999999999999
Monsters, Inc. (2001), Similarity Score: 0.9999999999999999
Moana (2016), Similarity Score: 0.9999999999999999
Emperor's New Groove, The (2000), Similarity Score: 0.9999999999999999
Adventures of Rocky and Bullwinkle, The (2000), Similarity Score: 0.9999999999999999

Collaborative-based filtering:
Similar movies to 'Toy Story (1995)':
Return of Jafar, The (1994), Similarity Score: 0.03525721771804848
House II: The Second Story (1987), Similarity Score: 0.03501318445782219
Return of the One-Armed Swordsman (1969), Similarity Score: 0.033687987245898235
Returner (Ritaanaa) (2002), Similarity Score: 0.03355050483326334
3 Days to Kill (2014), Similarity Score: 0.0331875671818464
